# Aşama 1) Veri Temizliği ve Düzenleme

In [80]:
import pandas as pd
import re

In [231]:
df_magaza = pd.read_excel('Ek_1_MagazaVerileri.xlsx', sheet_name="Mağazalar")

In [233]:
df_ortaklik_payi = pd.read_excel('Ek_1_MagazaVerileri.xlsx', sheet_name="Ortaklık Payı")

In [235]:
df_satis = pd.read_excel('Ek_1_MagazaVerileri.xlsx', sheet_name="Satış")

In [237]:
print(f"Magaza tablosunun büyüklüğü {df_magaza.shape}, Ortaklık payı tablosunun büyüklüğü {df_ortaklik_payi.shape}, Satış tablosunun büyüklüğü {df_satis.shape}")

Magaza tablosunun büyüklüğü (100, 8), Ortaklık payı tablosunun büyüklüğü (200, 5), Satış tablosunun büyüklüğü (72000, 8)


In [239]:
df_magaza.to_csv ("magaza.csv", sep=',') 
df_ortaklik_payi.to_csv("ortaklik_payi.csv", sep=',')
df_satis.to_csv('satis.csv', sep=',')

In [241]:
df_magaza_csv = pd.read_csv('magaza.csv')
df_ortaklik_payi_csv = pd.read_csv('ortaklik_payi.csv')
df_satis_csv = pd.read_csv('satis.csv')

In [243]:
# 1. SK. ifadesinden önce boşluk ekleme
pattern_sk = r'([A-Z0-9\.]+)(SK\.)'
df_magaza_csv['Mağaza Adresi'] = df_magaza_csv['Mağaza Adresi'].apply(lambda x: re.sub(pattern_sk, r'\1 \2', x))

# 2. no: ifadesinden sonra gelen rakam ile büyük harf arasına / ekleme
pattern_no = r'(no:\d+)\s([A-Z])\b'
df_magaza_csv['Mağaza Adresi'] = df_magaza_csv['Mağaza Adresi'].apply(lambda x: re.sub(pattern_no, r'\1/\2', x))

# 3. Son kelimeden önce "/" ekleme
df_magaza_csv['Mağaza Adresi'] = df_magaza_csv['Mağaza Adresi'].apply(lambda x: x.strip())  # Baştaki ve sondaki boşlukları temizle
df_magaza_csv['Mağaza Adresi'] = df_magaza_csv['Mağaza Adresi'].apply(lambda x: ' / '.join(x.rsplit(' ', 1)))  # Son kelimeden önce "/" ekle

# 4. Fazladan boşlukları temizleme (opsiyonel)
df_magaza_csv['Mağaza Adresi'] = df_magaza_csv['Mağaza Adresi'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))

In [245]:
print(df_magaza_csv['Mağaza Adresi'][0])

BAGCILAR MH. 1207. SK. STAR PLAS OFFICE C-BLOK APT. no:1/C BAGLAR / DIYARBAKIR


In [247]:
df_magaza_csv.to_csv ("magaza.csv", sep=',')

In [249]:
df_magaza_csv.columns

Index(['Unnamed: 0', 'Mağaza Kodu', 'Mağaza İsmi', 'Mağaza Tipi',
       'Personel Sayısı', 'Açılış Tarihi', 'Güncel Aylık Kira (TL)',
       'Mağaza Alanı (m2)', 'Mağaza Adresi'],
      dtype='object')

# Aşama 2) Veriyi PostgreSQL Veritabanına Kaydetme

In [ ]:
#1 Mağaza tablosunu PostgreSQL veritabanı içerisine alma

In [290]:
from sqlalchemy import create_engine
import pandas as pd
import psycopg2

In [284]:
df = pd.read_csv('magaza.csv', index_col=0)

In [292]:
conn = psycopg2.connect(
    dbname="mapit",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

# Verileri PostgreSQL'e aktar
cursor = conn.cursor()
for index, row in df.iterrows():
    cursor.execute(
        """
        INSERT INTO magaza ("Mağaza Kodu", "Mağaza İsmi", "Mağaza Tipi", "Personel Sayısı", "Açılış Tarihi", "Güncel Aylık Kira (TL)", "Mağaza Alanı (m2)", "Mağaza Adresi")
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (row["Mağaza Kodu"], row["Mağaza İsmi"], row["Mağaza Tipi"], row["Personel Sayısı"], row["Açılış Tarihi"], row["Güncel Aylık Kira (TL)"], row["Mağaza Alanı (m2)"], row["Mağaza Adresi"])
    )
conn.commit()
cursor.close()
conn.close()

In [294]:
#2 Ortaklık payı ve satış tablolarını PostgreSQL veritabanına alma

In [318]:
# CSV dosyasını oku
df = pd.read_csv('ortaklik_payi.csv')

conn = psycopg2.connect(
    dbname="mapit",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

cursor = conn.cursor()
for index, row in df.iterrows():
    cursor.execute(
        """
        INSERT INTO ortaklik_payi ("magaza_kodu", "magaza_ismi", "ortak_id", "ortak_adi_soyadi", "ortaklik_payi")
        VALUES (%s, %s, %s, %s, %s)
        """,
        (row["Mağaza Kodu"], row["Mağaza İsmi"], row["Ortak ID"], row["Ortak Adı Soyadı"], row["Ortalık Payı (%)"])
    )
conn.commit()
cursor.close()
conn.close()

In [322]:
df = pd.read_csv('satis.csv')
df.columns = ['id', 'magaza_kodu', 'urun_adi', 'urun_cesidi', 'urun_segmenti', 'adet', 'satis_tutari_tl', 'yil', 'ay']

import psycopg2
from psycopg2 import OperationalError

try:
    # PostgreSQL bağlantısı oluştur
    conn = psycopg2.connect(
        dbname="mapit",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()

    # Verileri ekle
    for index, row in df.iterrows():
        cursor.execute(
            """
            INSERT INTO satis (magaza_kodu, urun_adi, urun_cesidi, urun_segmenti, adet, satis_tutari_tl, yil, ay)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (row["magaza_kodu"], row["urun_adi"], row["urun_cesidi"], row["urun_segmenti"], 
             row["adet"], row["satis_tutari_tl"], row["yil"], row["ay"])
        )
    conn.commit()

except OperationalError as e:
    print(f"Bağlantı hatası: {e}")

finally:
    # Bağlantıyı kapat
    if conn:
        cursor.close()
        conn.close()

In [327]:
#Tablo sütunlarını TR karakter yapma

In [335]:
import psycopg2
import pandas as pd

# PostgreSQL bağlantısı oluştur
conn = psycopg2.connect(
    dbname="mapit",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

# Verileri çek
query = "SELECT * FROM magaza"
df = pd.read_sql(query, conn)

# CSV dosyasına kaydet
df.to_csv('C:/Users/onurc/Mapit/magaza_cleaned.csv', index=False, encoding='utf-8')

# Bağlantıyı kapat
conn.close()

C:\Users\onurc\AppData\Local\Temp\ipykernel_19288\594545847.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [337]:
import psycopg2
import pandas as pd

# PostgreSQL bağlantısı oluştur
conn = psycopg2.connect(
    dbname="mapit",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

# Verileri çek
query = "SELECT * FROM ortaklik_payi"
df = pd.read_sql(query, conn)

# CSV dosyasına kaydet
df.to_csv('C:/Users/onurc/Mapit/ortaklik_payi_cleaned.csv', index=False, encoding='utf-8')

# Bağlantıyı kapat
conn.close()

C:\Users\onurc\AppData\Local\Temp\ipykernel_19288\3634405344.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [339]:
import psycopg2
import pandas as pd

# PostgreSQL bağlantısı oluştur
conn = psycopg2.connect(
    dbname="mapit",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

# Verileri çek
query = "SELECT * FROM satis"
df = pd.read_sql(query, conn)

# CSV dosyasına kaydet
df.to_csv('C:/Users/onurc/Mapit/satis_cleaned.csv', index=False, encoding='utf-8')

# Bağlantıyı kapat
conn.close()

C:\Users\onurc\AppData\Local\Temp\ipykernel_19288\3921739095.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [341]:
df = pd.read_csv('magaza_cleaned.csv')

In [347]:
import re

def adresi_ayir(adres):
    # Bina numarasını bul
    bina_no = re.search(r'no:\s*\S+', adres)
    bina_no = bina_no.group(0) if bina_no else None

    # Sokak bilgisini bul
    sokak = re.search(r'\d+\.\s*SK\.', adres)
    sokak = sokak.group(0) if sokak else None

    # Mahalle bilgisini bul
    mahalle = re.search(r'[A-ZÇĞİÖŞÜ]+ MH\.', adres)
    mahalle = mahalle.group(0) if mahalle else None

    # İlçe ve il bilgisini bul
    ilce_il = re.split(r'\s*/\s*', adres)
    ilce = ilce_il[-2].strip() if len(ilce_il) > 1 else None
    il = ilce_il[-1].strip() if len(ilce_il) > 1 else None

    return pd.Series({
        "bina_no": bina_no,
        "sokak": sokak,
        "mahalle": mahalle,
        "ilce": ilce,
        "il": il
    })

In [351]:
# Adresleri ayrıştır ve yeni sütunlar olarak ekle
df[['bina_no', 'sokak', 'mahalle', 'ilce', 'il']] = df['magaza_adresi'].apply(adresi_ayir)

# İlk 5 satırı göster
df.head()

,magaza_kodu,magaza_ismi,magaza_tipi,personel_sayisi,acilis_tarihi,guncel_aylik_kira_tl,magaza_alani_m2,magaza_adresi,bina_no,sokak,mahalle,ilce,il
0,101,M101,Tip 2,8,2004-05-15,37088.0,976.0,BAGCILAR MH. 1207. SK. STAR PLAS OFFICE C-BLOK...,no:1/C,1207. SK.,BAGCILAR MH.,C BAGLAR,DIYARBAKIR
1,102,M102,Tip 4,18,2005-12-23,57474.0,1854.0,BAGLARBASI MH. ATATÜRK SK. TELESERVICE APT. no...,no:134,None,BAGLARBASI MH.,BAGLARBASI MH. ATATÜRK SK. TELESERVICE APT. no...,ISTANBUL
2,103,M103,Tip 2,11,2013-05-28,58960.0,1474.0,AYDINEVLER MH. SITELER YOLU SK. HILLTOWN AVM A...,no:1,None,AYDINEVLER MH.,AYDINEVLER MH. SITELER YOLU SK. HILLTOWN AVM A...,ISTANBUL
3,104,M104,Tip 7,12,2011-11-25,52836.0,1428.0,AYDINEVLER MH. SITELER YOLU SK. no:1/A MALTEPE...,no:1/A,None,AYDINEVLER MH.,A MALTEPE,ISTANBUL
4,105,M105,Tip 6,11,2018-02-14,39468.0,1012.0,KONACIK MH. CEVAHIR SK. B APT. no:9/B BODRUM /...,no:9/B,None,KONACIK MH.,B BODRUM,MUGLA


# 3) Veri Tiplerinin Kontrolü ve Dönüşümü

In [426]:
import pandas as pd

# Satış tablosunu oku
df_satis = pd.read_csv("satis_cleaned.csv")

# magaza_kodu sütunundaki "M" harfini kaldır
df_satis['magaza_kodu'] = df_satis['magaza_kodu'].str.replace('M', '', regex=False)

# Sonucu kontrol et
print(df_satis['magaza_kodu'].head())

0    101
1    101
2    101
3    101
4    101
Name: magaza_kodu, dtype: object


In [428]:
# magaza_kodu sütununu int'e dönüştür
df_satis['magaza_kodu'] = df_satis['magaza_kodu'].astype(int)

# Sonucu kontrol et
print(df_satis['magaza_kodu'].dtype)
print(df_satis['magaza_kodu'].head())

int64
0    101
1    101
2    101
3    101
4    101
Name: magaza_kodu, dtype: int64


In [430]:
# Yeni CSV dosyasına kaydet
df_satis.to_csv('satis_cleaned_string.csv', index=False, encoding='utf-8')

print("Güncellenmiş tablo 'satis_cleaned_numeric_magaza_kodu.csv' dosyasına kaydedildi.")

Güncellenmiş tablo 'satis_cleaned_numeric_magaza_kodu.csv' dosyasına kaydedildi.


In [434]:
import psycopg2
import pandas as pd
from psycopg2 import sql

# CSV dosyasını oku
df = pd.read_csv("magaza_cleaned_string.csv")

# PostgreSQL bağlantısı oluştur
conn = psycopg2.connect(
    dbname="mapit",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Tabloyu oluştur (eğer daha önce oluşturulmadıysa)
create_table_query = """
CREATE TABLE IF NOT EXISTS magaza (
    magaza_kodu INT PRIMARY KEY,
    magaza_ismi TEXT,
    magaza_tipi TEXT,
    personel_sayisi INT,
    acilis_tarihi DATE,
    guncel_aylik_kira_tl FLOAT,
    magaza_alani_m2 FLOAT,
    magaza_adresi TEXT
);
"""
cursor.execute(create_table_query)
conn.commit()

# Verileri tabloya ekle
for index, row in df.iterrows():
    insert_query = sql.SQL("""
    INSERT INTO magaza (magaza_kodu, magaza_ismi, magaza_tipi, personel_sayisi, acilis_tarihi, guncel_aylik_kira_tl, magaza_alani_m2, magaza_adresi)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """)
    cursor.execute(insert_query, (
        row['magaza_kodu'], row['magaza_ismi'], row['magaza_tipi'], row['personel_sayisi'],
        row['acilis_tarihi'], row['guncel_aylik_kira_tl'], row['magaza_alani_m2'], row['magaza_adresi']
    ))

# Değişiklikleri kaydet ve bağlantıyı kapat
conn.commit()
cursor.close()
conn.close()

print("CSV dosyası PostgreSQL tablosuna aktarıldı.")

CSV dosyası PostgreSQL tablosuna aktarıldı.


In [444]:
import psycopg2
import pandas as pd
from psycopg2 import sql

# CSV dosyasını oku
df = pd.read_csv("ortaklik_payi_cleaned_string.csv")

# Sütun isimlerini kontrol et ve gerekirse güncelle
df.columns = ['magaza_kodu', 'magaza_ismi', 'ortak_id', 'ortak_adi_soyadi', 'ortaklik_payi', 'id']

# PostgreSQL bağlantısı oluştur
conn = psycopg2.connect(
    dbname="mapit",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Tabloyu oluştur (eğer daha önce oluşturulmadıysa)
create_table_query = """
CREATE TABLE IF NOT EXISTS ortaklik_payi (
    magaza_kodu INT,
    magaza_ismi TEXT,
    ortak_id INT,
    ortak_adi_soyadi TEXT,
    ortaklik_payi FLOAT,
    id INT PRIMARY KEY
);
"""
cursor.execute(create_table_query)
conn.commit()

# Verileri tabloya ekle
for index, row in df.iterrows():
    insert_query = sql.SQL("""
    INSERT INTO ortaklik_payi (magaza_kodu, magaza_ismi, ortak_id, ortak_adi_soyadi, ortaklik_payi, id)
    VALUES (%s, %s, %s, %s, %s, %s)
    """)
    cursor.execute(insert_query, (
        row['magaza_kodu'], row['magaza_ismi'], row['ortak_id'], row['ortak_adi_soyadi'], row['ortaklik_payi'], row['id']
    ))

# Değişiklikleri kaydet ve bağlantıyı kapat
conn.commit()
cursor.close()
conn.close()

print("Ortaklık Payı tablosu PostgreSQL'e aktarıldı.")

Ortaklık Payı tablosu PostgreSQL'e aktarıldı.


In [442]:
import psycopg2
import pandas as pd
from psycopg2 import sql

# CSV dosyasını oku
df = pd.read_csv("satis_cleaned_string.csv")

# Sütun isimlerini kontrol et ve gerekirse güncelle
df.columns = ['id', 'magaza_kodu', 'urun_adi', 'urun_cesidi', 'urun_segmenti', 'adet', 'satis_tutari_tl', 'yil', 'ay']

# PostgreSQL bağlantısı oluştur
conn = psycopg2.connect(
    dbname="mapit",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Tabloyu oluştur (eğer daha önce oluşturulmadıysa)
create_table_query = """
CREATE TABLE IF NOT EXISTS satis (
    id INT PRIMARY KEY,
    magaza_kodu INT,
    urun_adi TEXT,
    urun_cesidi TEXT,
    urun_segmenti TEXT,
    adet INT,
    satis_tutari_tl FLOAT,
    yil INT,
    ay INT
);
"""
cursor.execute(create_table_query)
conn.commit()

# Verileri tabloya ekle
for index, row in df.iterrows():
    insert_query = sql.SQL("""
    INSERT INTO satis (id, magaza_kodu, urun_adi, urun_cesidi, urun_segmenti, adet, satis_tutari_tl, yil, ay)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """)
    cursor.execute(insert_query, (
        row['id'], row['magaza_kodu'], row['urun_adi'], row['urun_cesidi'], row['urun_segmenti'],
        row['adet'], row['satis_tutari_tl'], row['yil'], row['ay']
    ))

# Değişiklikleri kaydet ve bağlantıyı kapat
conn.commit()
cursor.close()
conn.close()

print("Satış tablosu PostgreSQL'e aktarıldı.")

Satış tablosu PostgreSQL'e aktarıldı.
